# 2

In [1]:
from load_datasets import train_val_generator, val_generator, train_generator, test_generator, X_train
from helper_functions import flatten_list
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [20]:
def model_cnn_2_classes(
    convolution_layers = [],
    mlp_layers = [],
    *,
    flatten_layer = [Flatten(name='flatten_layer')],
    name: str = "cnn_model_2_classes",
    optimizer=Adam(learning_rate=0.001),
    metrics=["acc"],
):
    model = Sequential(convolution_layers + flatten_layer + mlp_layers + [Dense(1, activation='sigmoid', name='output_layer')], name=name)
    model.compile(
        loss="binary_crossentropy", optimizer=optimizer, metrics=metrics
    )
    model.summary()
    return model

In [9]:
X_train.shape[1:]

(32, 32, 3)

In [32]:
input_shape = X_train.shape[1:]
kernel_numbers = [32, 64, 128, 128]
model_1__cnn_layers = flatten_list([
    [
        Conv2D(
            kernel_number,
            padding="same",
            kernel_size=(3, 3),
            activation="relu",
            kernel_initializer="he_normal",
            input_shape=(32, 32, 3),
        ),
        MaxPooling2D(pool_size=(2, 2), strides=2),
    ]
    for kernel_number in kernel_numbers
])
model_1__mlp_layers = [Dropout(.3), Dense(512, activation='relu', kernel_initializer='he_normal')]
model_1__cnn_layers, model_1__mlp_layers

([<keras.layers.convolutional.Conv2D at 0x29006348b20>,
  <keras.layers.core.dense.Dense at 0x29006323340>])

In [35]:
model_1 = model_cnn_2_classes(model_1__cnn_layers, model_1__mlp_layers, name='model_1')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_56 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d_56 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_57 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_57 (MaxPoolin  (None, 8, 8, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_58 (Conv2D)          (None, 8, 8, 128)         73856     
                                                                 
 max_pooling2d_58 (MaxPoolin  (None, 4, 4, 128)        0   